In [1]:
"""DE Test Case."""
from pathlib import Path
import pandas as pd
import pomato

In [2]:
# wdir = Path("/examples/") # Change to local copy of examples folder
wdir = Path.cwd()
mato = pomato.POMATO(wdir=wdir, options_file="profiles/de.json")
mato.load_data('data_input/DE_2020.zip')


INFO - Creating Folder Structure
INFO - creating folder data_output
INFO - creating folder data_temp
INFO - creating folder data_temp/julia_files
INFO - creating folder data_temp/julia_files/data
INFO - creating folder data_temp/julia_files/results
INFO - creating folder data_temp/julia_files/cbco_data
INFO - Initializing DataObject
INFO - Initializing the GridTopology....
INFO - Initializing MarketModel...
INFO - Initializing the FBMCModule....
INFO - Initializing the GridTopology....
INFO - POMATO initialized!
INFO - Loading data from zipped data archive
WARNING - Not complete list of expected input data found. See .missing_data
INFO - Validating Input Data...
WARNING - Data validation completed with warnings. See the .data_validation_report.
WARNING - Some data was initialized empty. See model_validation_report.
WARNING - Some data missing or contained NaNs. See model_validation_report.
INFO - Calculating grid parameters!
INFO - The network consists of 28 components. Setting slack f

In [3]:
# Create the grid representation, the options are specified in the 
# json file supplied in the instantiation. 
# In this case the model runs a zonal pricing with NTC to neighboring countries
# and subsequent redispatch.
mato.create_grid_representation()
mato.run_market_model()

INFO - Start-Time: 12:45:32
Starting Julia Process... INFO - Loading Gurobi...
INFO - Initialized
Starting Julia Process... -INFO - reading from file c:\users\riw\documents\repositories\pomato\pomato/daemon_market_model.json
INFO - Done with Initialization. Starting daemon process market_model!
INFO - Process ready!
Processing... \INFO - Loading Gurobi...
INFO - Starting with market_model
INFO - Run market model, including redispatch
INFO - No logfile Found, logging only to console.
INFO - Read Model Data..
INFO - Reading Model Data from: c:\Users\riw\Documents\repositories\pomato\examples/data_temp/julia_files/data/
INFO - Data Prepared
Processing... |INFO - Creating Results Folder
INFO - Initializing Market Model for timestep t0001...
INFO - Academic license - for non-commercial use only
INFO - Adding Variables and Expressions..
INFO - Add Base Model
INFO - Adding NTC Constraints...
INFO - Adding Objective Function...
INFO - Solving...
INFO - Gurobi Optimizer version 9.0.2 build v9.0

In [4]:
# Return the results, in this case two: 
# market result (uniform pricing, no network constraints)
# and redispatch (to account for N-0 network feasibility)
market_result, redisp_result = mato.data.return_results()


In [5]:
# Check for overloaded lines in the market and redispatch results
n0_m, _ = market_result.overloaded_lines_n_0()
print("Number of N-0 Overloads in market result: ", len(n0_m))
n0_r, _  = redisp_result.overloaded_lines_n_0()
print("Number of N-0 Overloads after redispatch: ", len(n0_r))

Number of N-0 Overloads in market result:  127
Number of N-0 Overloads after redispatch:  21


In [6]:
# Generation comparison between Market Result and Redispatch.
# Redispatch is calculated G_redispatch - G_market as delta
# The absolute delta represents the total redispatched energy
gen = pd.merge(market_result.data.plants[["plant_type", "fuel", "technology", "g_max", "node"]],
               market_result.G, left_index=True, right_on="p")
gen = pd.merge(gen, redisp_result.G, on=["p", "t"], suffixes=("_market", "_redispatch"))
gen["delta"] = gen["G_redispatch"] - gen["G_market"]
gen["delta_abs"] = gen["delta"].abs()
print("Redispatched energy per hour (abs) [MWh] ", gen.delta_abs.sum()/len(gen.t.unique()))

# Many of these functions are implemented as methods of the Result class
gen_2 = redisp_result.redispatch()
print("Redispatched energy per hour (abs) [MWh] ", gen_2.delta_abs.sum()/len(gen_2.t.unique()))



Redispatched energy per hour (abs) [MWh]  639.0308382572357
Redispatched energy per hour (abs) [MWh]  639.0308382572357


In [10]:
# Create Geo Plot. DISCLAiMER: The reported prices are the dual 
# in the redispatch result, thus including costs for redispatch.
mato.visualization.create_geo_plot(redisp_result, show_prices=False, show_plot=False,show_redispatch=True, filepath=mato.wdir.joinpath("redispatch.html"))


In [11]:
# Create visualization of the generation schedule in the market result. 
mato.visualization.create_generation_plot(market_result, show_plot=False, filepath=mato.wdir.joinpath("market_results.html"))



In [9]:
# 
mato._join_julia_instances()


INFO - EXIT
